# Imports

In [30]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from IPython.display import display

# Pre-process Data

In [32]:
df_raw = pd.read_csv("../data/ObesityDataSet_raw_and_data_sinthetic.csv")
df_raw.columns = ['gender', 'age', 'height', 'weight', 'family_history_with_overweight',
'frequent_consumption_of_high_caloric_food', 'frequency_of_consumption_of_vegetables', 'number_of_main_meals', 'consumption_of_food_between_meals', 'smoke', 'consumption_of_water_daily', 'calories_consumption_monitoring', 'physical_activity_frequency', 'time_using_technology_devices',
'consumption_of_alcohol', 'transportation_used', 'obesity']
df_proc = df_raw.copy()
df_proc["age"] = df_proc["age"].round(0).astype(int)
df_proc["height"] = df_proc["height"].round(2)
df_proc["weight"] = df_proc["weight"].round(1)
df_proc["frequency_of_consumption_of_vegetables"] = df_proc["frequency_of_consumption_of_vegetables"].round(0).astype(int)
df_proc["number_of_main_meals"] = df_proc["number_of_main_meals"].round(0).astype(int)
df_proc["consumption_of_water_daily"] = df_proc["consumption_of_water_daily"].round(0).astype(int)
df_proc["physical_activity_frequency"] = df_proc["physical_activity_frequency"].round(0).astype(int)
df_proc["time_using_technology_devices"] = df_proc["time_using_technology_devices"].round(0).astype(int)

print(df_raw.shape, df_proc.shape)
display(df_proc.describe())
# display(df_proc.head(20))
# display(df_proc.tail(20))
display(df_proc)
display(df_raw)

# ordinal: consumption_of_food_between_meals, consumption_of_alcohol, obesity
# one hot: family_history_with_overweight, frequent_consumption_of_high_caloric_food, smoke, calories_consumption_monitoring, transportation_used, 

(2111, 17) (2111, 17)


,age,height,weight,frequency_of_consumption_of_vegetables,number_of_main_meals,consumption_of_water_daily,physical_activity_frequency,time_using_technology_devices
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.315964,1.701620,86.585978,2.423496,2.687826,2.014685,1.006632,0.664614
std,6.357078,0.093368,26.191165,0.583905,0.809680,0.688616,0.895462,0.674009
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,20.000000,1.630000,65.450000,2.000000,3.000000,2.000000,0.000000,0.000000
50%,23.000000,1.700000,83.000000,2.000000,3.000000,2.000000,1.000000,1.000000
75%,26.000000,1.770000,107.450000,3.000000,3.000000,2.000000,2.000000,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


,gender,age,height,weight,family_history_with_overweight,frequent_consumption_of_high_caloric_food,frequency_of_consumption_of_vegetables,number_of_main_meals,consumption_of_food_between_meals,smoke,consumption_of_water_daily,calories_consumption_monitoring,physical_activity_frequency,time_using_technology_devices,consumption_of_alcohol,transportation_used,obesity
0,Female,21,1.62,64.0,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Normal_Weight
1,Female,21,1.52,56.0,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23,1.80,77.0,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Normal_Weight
3,Male,27,1.80,87.0,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Overweight_Level_I
4,Male,22,1.78,89.8,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,21,1.71,131.4,yes,yes,3,3,Sometimes,no,2,no,2,1,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,22,1.75,133.7,yes,yes,3,3,Sometimes,no,2,no,1,1,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,23,1.75,133.7,yes,yes,3,3,Sometimes,no,2,no,1,1,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24,1.74,133.3,yes,yes,3,3,Sometimes,no,3,no,1,1,Sometimes,Public_Transportation,Obesity_Type_III


,gender,age,height,weight,family_history_with_overweight,frequent_consumption_of_high_caloric_food,frequency_of_consumption_of_vegetables,number_of_main_meals,consumption_of_food_between_meals,smoke,consumption_of_water_daily,calories_consumption_monitoring,physical_activity_frequency,time_using_technology_devices,consumption_of_alcohol,transportation_used,obesity
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [29]:
df_raw[df_raw['number_of_main_meals'] >= 4.0]

,gender,age,height,weight,family_history_with_overweight,frequent_consumption_of_high_caloric_food,frequency_of_consumption_of_vegetables,number_of_main_meals,consumption_of_food_between_meals,smoke,consumption_of_water_daily,calories_consumption_monitoring,physical_activity_frequency,time_using_technology_devices,consumption_of_alcohol,transportation_used,obesity
18,Female,30,1.71,82.0,yes,yes,3,4,Frequently,yes,1,no,0,0,no,Automobile,Overweight_Level_II
25,Male,20,1.60,50.0,yes,no,2,4,Frequently,yes,2,no,3,2,no,Public_Transportation,Normal_Weight
27,Female,23,1.60,52.0,no,yes,2,4,Frequently,no,2,no,2,1,Sometimes,Automobile,Normal_Weight
30,Male,29,1.77,83.0,no,yes,1,4,Frequently,no,3,no,0,1,no,Motorbike,Overweight_Level_I
34,Male,22,1.65,62.0,no,yes,2,4,Frequently,no,2,no,2,0,Sometimes,Public_Transportation,Normal_Weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,Male,31,1.73,110.7,yes,yes,2,4,Sometimes,no,2,no,1,0,no,Automobile,Obesity_Type_II
1603,Male,24,1.62,100.0,yes,yes,3,4,Sometimes,no,1,no,2,1,no,Public_Transportation,Obesity_Type_II
1627,Male,40,1.72,109.3,yes,yes,2,4,Sometimes,no,1,no,1,0,no,Automobile,Obesity_Type_II
1699,Male,40,1.71,108.0,yes,yes,2,4,Sometimes,no,1,no,2,0,no,Automobile,Obesity_Type_II


In [10]:
# source ref: https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

print(f"Features: {diabetes.feature_names}\n")
print(diabetes.DESCR)

Features: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum trigly